In [80]:
import pandas as pd 
players = pd.DataFrame(pd.read_json('players.json'))
events = pd.DataFrame(pd.read_json('events_England.json'))
teams = pd.read_json('teams.json')
events = events.loc[events['eventName'] ==  'Pass'] # I take only the pass
events.reset_index(drop = True,inplace = True)
events0 = events['tags']
lst = []
for rows in events0:
    for x in rows:
        y = x['id']
    lst.append(y)
d = {"SuccesId" : lst}
db = pd.DataFrame(d)
lst2 = [db,events]
events1 = pd.concat(lst2,axis = 1) # I extract and concatenate l'Id (1801 or 1802)
players = players.rename(columns={"wyId":"playerId"})
events2 = pd.merge(events1,players,on = "playerId") 
events3 = events2.groupby(['playerId'])['SuccesId'].count().reset_index() # I group by player and count the total passes
events3 = events3.loc[events3['SuccesId'].map(int) >= (events3['SuccesId'].mean()-events3['SuccesId'].std())] # I take only values >= mean - std
events4 = events2.loc[events2['SuccesId'].map(int) == 1801] # I take only completed pass
events4 = events4.groupby(['playerId'])['SuccesId'].count().reset_index() # I group by player and count the completed passes
events5= pd.merge(events3,events4,on = ["playerId"]).rename(columns={'SuccesId_x':'events','SuccesId_y' : 'accurate_events'})
lst1 = []
for i in range(len(events5.index)):
    x = ((events5['accurate_events'].iloc[i])/(events5['events'].iloc[i]))
    lst1.append(x)
d1 = {"percentege_of_accurate_events" : lst1}
db1 = pd.DataFrame(d1)
lst3 = [events5,db1]
events6 = pd.concat(lst3,axis = 1) # i estimate the ratio between completed passes and attempted passes.
events7 = events6.sort_values(by = 'percentege_of_accurate_events',ascending = False) 
events7  = events7.reset_index(drop = True)
events7 = events7.head(10) # i show  the 10 players with the highest ratio

In [81]:
events7

,playerId,events,accurate_events,percentege_of_accurate_events
0,9380,1204,1175,0.975914
1,8324,404,383,0.948020
2,8464,1584,1497,0.945076
3,8307,1391,1312,0.943206
4,135103,1288,1211,0.940217
5,7856,233,219,0.939914
6,7918,1057,992,0.938505
7,70086,2964,2779,0.937584
8,120339,834,781,0.936451
9,14808,1350,1263,0.935556
